### Note

Pre-processing code for adding Question-Answer format column to datasets.

### Set Hugging Face Token

In [1]:
token = #####

from huggingface_hub import login
login()

In [1]:
import os
import random
from datasets import load_dataset
import pandas as pd

CACHE_DIR = os.getcwd()+'/cache'

### Lima Dataset

In [2]:
dataset = load_dataset("GAIR/lima", cache_dir=CACHE_DIR)

In [3]:
train_df = dataset['train'].to_pandas()
test_df = dataset['train'].to_pandas()

In [4]:
train_df.head()

,conversations,source
0,[Can brain cells move? By movement I mean long...,stackexchange
1,[In our computer systems lecture we were intro...,stackexchange
2,[View tabular file such as CSV from command li...,stackexchange
3,[Slater type orbitals (STO) are considered to ...,stackexchange
4,"[Explain what ""git reset"" does. I come from a ...",stackexchange


In [5]:
def add_text_column(row):
    return f"###Question:\n{row['conversations'][0]}\n\n###Answer:\n{row['conversations'][1]}"

train_df['text'] = train_df.apply(lambda x: add_text_column(x), axis=1)
test_df['text'] = test_df.apply(lambda x: add_text_column(x), axis=1)

In [6]:
train_df.to_csv("ProcessedData/lima_train.csv", index=False)
test_df.to_csv("ProcessedData/lima_test.csv", index=False)

### SHP Dataset

In [2]:
dataset = load_dataset("stanfordnlp/shp", cache_dir=CACHE_DIR, split="train")

In [12]:
# Filter Data to keep max score row
train_df = dataset.to_pandas()

print("Number of Unique IDs:", train_df['post_id'].nunique())

train_df['max_score'] = train_df.apply(lambda x: x['score_B'] if x['labels']==0 else x['score_A'], axis=1)
train_df = train_df.sort_values(by='max_score', ascending=False).reset_index(drop=True)
train_df = train_df.drop_duplicates(subset=['post_id']).reset_index(drop=True)
train_df['human_ref'] = train_df.apply(lambda x: x['human_ref_A'] if x['labels']==1 else x['human_ref_B'], axis=1)
train_df = train_df[['post_id', 'domain', 'history', 'human_ref']]

Number of Unique IDs: 38993


In [13]:
train_df.head()

,post_id,domain,history,human_ref
0,gz5idy,explainlikeimfive_train,Explain like I'm five years old: Why do ships ...,The joke answer is so that the water doesn't h...
1,9durlp,explainlikeimfive_train,"Eli5: Caffeine has almost no calories, but see...",Caffeine works in two ways to make you feel th...
2,azzxtj,explainlikeimfive_train,Explain like I'm five years old: What makes a ...,"If you stab your belly with a knife, first you..."
3,lwmmm9,explainlikeimfive_train,Explain like I'm five years old: Why can your ...,Your brain is like a trash can. Throughout the...
4,q8cfc6,explainlikeimfive_train,Explain like I'm five years old: Why exactly i...,"You know when you're going on vacation, and yo..."


In [16]:
def add_text_column(row):
    return f"###Question:\n{row['history']}\n\n###Answer:\n{row['human_ref']}"

train_df['text'] = train_df.apply(lambda x: add_text_column(x), axis=1)

In [12]:
train_df[['domain', 'text']].to_csv("ProcessedData/SHP_train.csv", index=False)

### DPO Comparison Dataset

In [20]:
dataset = load_dataset("stanfordnlp/shp", cache_dir=CACHE_DIR, split="train")
train_df = dataset.to_pandas()

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [26]:
train_df['chosen'] = train_df.apply(lambda x: x['human_ref_A'] if x['labels']==1 else x['human_ref_B'], axis=1)
train_df['rejected'] = train_df.apply(lambda x: x['human_ref_A'] if x['labels']==0 else x['human_ref_B'], axis=1)

train_df = train_df.rename(columns={"history": "prompt"})

### Anthropic Dataset

In [2]:
# Download Anthropic to Cache folder
dataset_name = 'Anthropic/hh-rlhf'
dataset = load_dataset(dataset_name, split="train", cache_dir=CACHE_DIR)